### Assignment-5 : Keyboard optimization
- In this assignment , given a string , we will try to optimize the position of the keys in the keyboard such that we get minimal distance.
- We use simulated annealing optimization technique to perform optimzation.
- Inputs needed are : A string and a layout to start with
- Outputs : An animated graph that tells the best optimzation and current optimization

### Utility modules

In [1]:
%matplotlib inline
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as patches
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.animation import FuncAnimation
from collections import defaultdict
from collections import Counter
import random
import copy
import time

### Module to define the initial layout with the necessary parameters
- pos : To define the position of keys
- cost : To define the distance needed to type the key.
- index : Just defining the order of the keys

In [2]:
def init(layout,string):
    
    layout.keys['Space']['start']='Space'
    pos={} # Dictionary to store the coordinates of each letter
    hom={} # Dictionary to store the home array or the letter to start typing from
    cost={} # Dictionary to store the cost of typing each letter
    index={} # Dictionary to store the index or position of each letter in the above dictionaries
    char_index={} # Dictionary to store the index to the corresponding character
    travel_cost=0
    ind=0
    for i in layout.keys:
        pos[i]=layout.keys[i]['pos'] # Assigning position with the keys dictionary present in the layout 
        hom[i]=layout.keys[i]['start'] # Assigning hom position with the keys dictionary present in the layout
        index[i]=ind # Assigning the index of each letter
        char_index[ind]=i #Assigning each index its character
        ind+=1
    
    for i in pos:
        x1,y1=pos[i]
        x2,y2=pos[hom[i]]
        cost[i]=np.sqrt((x1-x2)**2+(y1-y2)**2) # Computing the Eucledian distance
        
    char_count = Counter(string)
    params=[pos,cost,index,char_index]
    
    for char, count in char_count.items():
        if char in layout.characters:
            travel_cost += sum(params[1][j] for j in layout.characters[char]) * count
        
    
    return([params,travel_cost]) # Returns the list that contains the params and travel cost 

### Function to interchange or swap two keys.
- To swap the keys , we use random module to choose two numbers and swap the corresponding keys
- We also change the travelling cost by subtracting with frequency*cost of the key

In [3]:
def new_layout(in_param,data):
    param = copy.deepcopy(in_param) #To copy the previous parameter
    params,travel_cost=param
    i, j = random.sample(range(len(params[0])), 2) #Choosing two keys randomly
    i,j=params[-1][i],params[-1][j]
    # Subtracting the cost if there is a frequency
    if(data[params[2][i]]>0):
        travel_cost-=data[params[2][i]]*(params[1][i])
    if(data[params[2][j]]>0):
        travel_cost-=data[params[2][j]]*(params[1][j])
    params[-1][j],params[-1][i]=i,j # Swapping
    l=[params[i] for i in range(len(params)-1)] #Having a list to swap all the parameters
    for k in l:
        _=k[i]
        k[i]=k[j]
        k[j]=_
    # Adding the cost if there is a frequency
    if(data[params[2][i]]>0):
        travel_cost+=data[params[2][i]]*(params[1][j])
    if(data[params[2][j]]>0):
        travel_cost+=data[params[2][j]]*(params[1][i])
    l.append(params[-1])
    return([l,travel_cost])

### Function to generate gaussian heatmap
- The module plt.imshow() takes a 2D array as input and assigns the values in the array to the points of the graph.
- So , we need to assign values to the 2D array such that it forms a heatmap.
- The following function does that
- sigma in the following function is the blending parameter.

In [4]:

def generate_circular_heatmap(lst,coordinates, grid_sizex,grid_sizey, sigma=0.35):

    x = np.linspace(-5, 16,grid_sizex)
    y = np.linspace(-1, 5, grid_sizey)
    X, Y = np.meshgrid(x, y)
    heatmap = np.zeros(X.shape)
    dat = (lst - np.min(lst)) / (np.max(lst) - np.min(lst))
    
    # coordinates = np.array(coordinates)
    for i, (cx, cy) in enumerate(coordinates):
        distance = np.sqrt((X - cx - 0.35) ** 2 + (Y - cy - 0.25) ** 2)
        heatmap += dat[i] * np.exp(-(distance ** 2) / (2 * sigma ** 2))
    
    return heatmap

### Function to compute cost for each iteration
- It takes in string and goes to the given layout to find the starting position of each character and returns the travel cost

In [5]:

def cost_fn(string, layout, params):
    travel_cost = 0
    char_count = Counter(string)

    for char, count in char_count.items():
        if char in layout.characters:
            travel_cost += sum(params[1][j] for j in layout.characters[char]) * count #Computes the cost

    return travel_cost


### Function to compute the frequency array to generate heatmap
- This is a constant array as frequency of a character won't change in a simulated annealing algorithm

In [6]:
def frequency_array(string, layout, param):
    params,_=param
    # Create a frequency counter for the characters in the string
    char_count = Counter(string)

    # Initialize the frequency array
    lst = np.zeros(len(params[0]), dtype=int)

    # Iterate over each character and its count
    for char, count in char_count.items():
        if char in layout.characters:
            for j in layout.characters[char]:
                lst[params[2][j]] += count  # Accumulate the count with the lst i.e frequency array

    return lst

### Simulated Annealing
- The algorithm takes two random keys , swaps their position
- Finds the cost of the swapped position and chooses to continue with that if the probaility is greater than the given temperature cooling process function.

In [8]:
def simulated_annealing(layout,string , initial_temp, cooling_rate, num_iterations,params,data):
    current_params ,current_out = params
    current_out = cost_fn(string,layout,current_params)
    best_param = copy.deepcopy(current_params)
    best_out = current_out
    
    temp = initial_temp
    outs = [current_out]
    best_params = [best_param.copy()]
    best_outs = [best_out]
    
    for i in range(num_iterations):
        # print("Old\n",current_params[0])
        new_params,new_out = new_layout([current_params.copy(),current_out],data)
        
        p = np.exp((current_out- new_out) / temp)
        # print(f"Prob[{i} = {p}")
        # print("Old\n",best_out[1])
        if new_out < current_out or random.random() < p:
            current_params = new_params.copy()
            current_out = new_out
            
            if current_out< best_out:
                best_param = copy.deepcopy(current_params)
                best_out = current_out
        # print("New\n",best_out[1])
        temp *= cooling_rate
        outs.append(current_out)
        best_params.append(best_param.copy())
        best_outs.append(best_out)

    return best_params, best_outs, outs

### Function to generate the keyboard animation
- Same function as used in previous assignment
- I have defined variables like rect_width , text_offset which are best suited for drawing the keyboard layouts after experimenting with multiple other values
- One can play around with these values to change the size of the keyboard and keys.

In [9]:
colorbar = None # A global variable to check the previously added colorbar
text_objects = {} # A global variable to check the previouly added texts

#Custom colormaps

colors = ['lightgrey', 'blue', 'green', 'yellow', 'red']
custom_cmap = LinearSegmentedColormap.from_list("CustomMap", colors, N=256)

def keyboard(layout, params, data, ax):
    global colorbar,text_objects
    pos, _, _, _ = params  # Accessing relevant params
    ax.set_facecolor("lightgrey")
    ax.set_xlim(-2, 17)
    ax.set_ylim(-1, 5)
    
    # Rectangle properties
    rect_width, rect_height = 0.8, 0.7
    text_offset_single = (0.2, 0.1)
    text_offset_multiple = (0.1, 0.15)
    # print(pos)
    for key in layout.keys:
        x, y = pos[key]
        if key in text_objects:
            text_objects[key].remove() #Removing the previously placed keys and adding new ones
        if len(key) == 1:
            text_objects[key] = ax.text(x + text_offset_single[0], y + text_offset_single[1], key.upper(), fontsize=13)
        else:
            text_objects[key] = ax.text(x + text_offset_multiple[0], y + text_offset_multiple[1], key.upper(), fontsize=5)
        
        rect = patches.Rectangle((x, y), rect_width, rect_height, edgecolor='black', facecolor='none')
        ax.add_patch(rect)

    # Generating the heatmap
    coord=params[0].values()
    heatmap=generate_circular_heatmap(data,coord,134,54)
    # Displaying the heatmap
    sm=ax.imshow(heatmap,extent=(-5, 16, -1, 5), origin='lower', cmap=custom_cmap, alpha=0.7) # Displaying the graph
    if colorbar is not None:
        colorbar.remove()
    
    # Add a new colorbar
    colorbar = plt.colorbar(sm, ax=ax, label='Intensity')
    


### Function to update the animated frames
- This calls the keyboard() function and updates the graph that contained the current/best output values

In [10]:
def update(frame,param,data,distances, cur_dist, distance_line, cur_dist_line,ax,layout,condt):
    # print("Hello")
    if(condt):
        keyboard(layout,param[frame],data,ax)
    distance_line.set_data(range(frame + 1), distances[:frame + 1])
    cur_dist_line.set_data(range(frame + 1), cur_dist[:frame + 1])
    return distance_line, cur_dist_line

### Function where the code executes all the above defined functions

In [11]:
def main(layout,string,condt):
    initial_temp = 100
    cooling_rate = 0.99 #Set it to be 0.99 as (0.99)^200 = 0.1 which would imply that it won't accept a lot of higher cost
    num_iterations = 20
    params=init(layout,string) #Initializing the layout
    data=frequency_array(string,layout,params) #Initializing the  frequency array
    #Creating a figure to show the plots
    fig, (ax1,ax2) = plt.subplots(2,1,figsize=(14,6))
    fig.suptitle("Simulated Annealing for Keyboard Problem",fontsize=15)
    # Calling Simulated annealing
    best_params, best_outs, cur_outs= simulated_annealing(layout,string, initial_temp, cooling_rate, num_iterations,params,data)
    if(best_outs[0]!=0): percent=((best_outs[0]-best_outs[-1])/(best_outs[0]))*100
    else: percent=0
    print("It got optimized by ",percent," %")
    # Setting thecurrent_out and best_out plot
    if(not condt):
        keyboard(layout,best_params[-1],data,ax1)
    ax2.set_xlim(0, num_iterations)
    ax2.set_ylim(min(cur_outs) * 0.9, max(cur_outs) * 1.1)
    ax2.set_title("Best Distance over Iterations",fontsize=15)
    ax2.set_xlabel("Iteration",fontsize=15)
    ax2.set_ylabel("Distance",fontsize=15)
    distance_line, = ax2.plot([], [], 'r-')
    cur_dist_line, = ax2.plot([], [], 'g-')

    # Create the animation
    anim = FuncAnimation(fig, update, frames=range(0, num_iterations, 1), 
                         fargs=(best_params,data,best_outs,cur_outs,distance_line,cur_dist_line,ax1,layout,condt),
                         interval=50, blit=True, repeat=False)
    return anim
    

### Inputs/Outputs :
- The boolean variable condt is used to determine whether to show the animation of keyboard or not
- If condt==**True** , it will display the animation of the keyboard key changing also
- Else , it just displays the final layout of the keyboard

In [12]:
import colemak_layout as clt
st="""Theqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqs) explaining your approach. Note that trying to squeeze in a lot of text or figures using small font does not qualify here: you are expected to be brief. In case you absolutely cannot fit in 3 pages, it is better to have a clear explanation of exactly what is there in the extra pages than to make it unreadable."""
st=st.replace("\n","")
st=st.replace(" ","")
st="qwfpg"
anim=main(clt,st,condt=False)
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128
from IPython.display import HTML
HTML(anim.to_jshtml())

<IPython.core.display.Javascript object>

It got optimized by  0.0  %


In [ ]:
import qwerty_layout as dlt
st="qwertyuiop"
st=st.replace("\n","")
st=st.replace(" ","")
anim=main(dlt,st,condt=False)
from IPython.display import HTML
HTML(anim.to_jshtml())